<a href="https://colab.research.google.com/github/anindabitm/Zindi_hack/blob/master/Zindi_hack3_nb_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

In [0]:
#Mount google drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Zindi_Hack/3/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# import statements
import os
import sys
import csv
import shutil
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import random
%matplotlib inline

In [0]:
#Install fastai2
!pip install fastai2

In [0]:
#import fastai modules
from fastai2.torch_basics import *
from fastai2.basics import *
from fastai2.data.all import *
from fastai2.callback.all import *
from fastai2.vision.all import *
from fastai2.test_utils import *
from fastai2.vision.core import *

In [0]:
#seed everything
seed = 42
set_seed(42)
torch.backends.cudnn.benchmark = False
# python RNG
random.seed(seed)
# pytorch RNGs
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
# numpy RNG
np.random.seed(seed)

In [0]:
!ls

images			  submission_file_tta.csv
images.zip		  submission_file_tta_cv.csv
sample_sub_v2.csv	  submission_file_tta_cv_dense169.csv
submission_file.csv	  submission_file_tta_cv_mixup.csv
submission_file_tta1.csv  train_labels.csv


In [0]:
!unzip images.zip

In [0]:
import glob
fn=glob.glob('./images/*.*')
len(fn)

1817

In [0]:
train=pd.read_csv('./train_labels.csv')
train.shape

(1308, 2)

In [0]:
train.head()

,image,target
0,tfymlmdkpzkqdjhdxyhnoeuqszxphw.jpg,0
1,rxgismgsvmaayzjarbfjaljhqmpbrt.jpg,1
2,uuzshfrhkgrkolhwdvliqauzulurnz.jpg,0
3,mjspxsagzusaznvnyxgamtrlqkqklp.jpg,0
4,rlbmuajgezfiddjzlyeoupxpqubkpt.jpg,1


In [0]:
sub=pd.read_csv('./sample_sub_v2.csv')
print(sub.shape)
sub.head()

(509, 2)


,image,target
0,aadawlxbmapqrblgxyzarhjasgiobu.png,0
1,abpxvdfyhaaohzcrngcucmhffwizxs.jpg,0
2,aclkjfvackiieiznzfcwienplielrj.jpg,0
3,aelkivmayxgsdjosiibfgmkvfrjvjk.jpg,0
4,aelzzshpfxerelefnkatpczktuxjln.jpg,0


In [0]:
#Creating cross-validation framework
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
X=train['image'].copy()
y=train['target'].copy()
fold = 0
for train_index, test_index in skf.split(X, y):
  fold+= 1
  print('In fold',fold)
  print("TRAIN LENGTH:", len(train_index), "VALIDATION LENGTH:", len(test_index))
  train[f'fold_{fold}_valid']=False
  train.loc[test_index,f'fold_{fold}_valid']=True

In fold 1
TRAIN LENGTH: 1046 VALIDATION LENGTH: 262
In fold 2
TRAIN LENGTH: 1046 VALIDATION LENGTH: 262
In fold 3
TRAIN LENGTH: 1046 VALIDATION LENGTH: 262
In fold 4
TRAIN LENGTH: 1047 VALIDATION LENGTH: 261
In fold 5
TRAIN LENGTH: 1047 VALIDATION LENGTH: 261


In [0]:
#Create dataloader 
def dataloader(fold):
  dls = ImageDataLoaders.from_df(train, path='.', folder='images', seed=42, item_tfms=Resize(224),batch_tfms = [*aug_transforms(size=224), Normalize.from_stats(*imagenet_stats)],bs=8,num_workers=0) 
  return dls

In [0]:
#Fairly balanced dataset
train['target'].value_counts()

0    664
1    644
Name: target, dtype: int64

In [0]:
#Cross validation and test time augmentation for robust prediction
test=sub.copy()
test.drop(['target'],axis=1,inplace=True)

final_preds1=np.zeros((sub.shape[0],sub.shape[1]))
fold = 0
for fold in range(5):
  fold+=1
  print('In fold:',fold)
  dls=dataloader(fold)
  learn1 = cnn_learner(dls,densenet201,metrics=[accuracy])
  learn1.fine_tune(10)
  test_dl=learn1.dls.test_dl(test)
  preds, _ = learn1.tta(dl=test_dl)
  print('Prediction completed in fold: {}'.format(str(fold)))
  final_preds1+=preds.numpy()
  
final_preds1=final_preds1/5

In fold: 1


Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/checkpoints/densenet201-c1103571.pth


epoch,train_loss,valid_loss,accuracy,time
0,0.637251,0.276276,0.923372,07:35


epoch,train_loss,valid_loss,accuracy,time
0,0.375437,0.125009,0.973180,01:04
1,0.319370,0.163981,0.965517,01:04
2,0.258610,0.204441,0.942529,01:04
3,0.176755,0.128468,0.957854,01:07
4,0.187185,0.084749,0.973180,01:04
5,0.171441,0.065768,0.973180,01:04
6,0.076938,0.070019,0.980843,01:04
7,0.118268,0.110251,0.965517,01:04
8,0.070507,0.068696,0.977012,01:04
9,0.093235,0.063354,0.984674,01:04


Prediction completed in fold: 1
In fold: 2


...

epoch,train_loss,valid_loss,accuracy,time
0,0.318422,0.130879,0.957854,01:03
1,0.381169,0.137703,0.950192,01:03
2,0.278567,1.087278,0.869732,01:03
3,0.213356,0.087790,0.969349,01:04
4,0.163621,0.045368,0.992337,01:04
5,0.171620,0.102962,0.969349,01:04
6,0.141315,0.059070,0.984674,01:04
7,0.092802,0.050704,0.984674,01:04
8,0.053216,0.041677,0.988506,01:04
9,0.052593,0.042985,0.988506,01:04


Prediction completed in fold: 4
In fold: 5


epoch,train_loss,valid_loss,accuracy,time
0,0.566201,0.190782,0.950192,00:55


epoch,train_loss,valid_loss,accuracy,time
0,0.444535,0.135635,0.961686,01:04
1,0.331684,0.082716,0.977012,01:04
2,0.271019,0.128941,0.973180,01:03
3,0.304309,0.097498,0.969349,01:03
4,0.152406,0.109241,0.961686,01:03
5,0.168653,0.109122,0.977012,01:03
6,0.119191,0.071969,0.984674,01:04
7,0.138663,0.071029,0.988506,01:04
8,0.044581,0.057937,0.988506,01:04
9,0.099029,0.056997,0.988506,01:04


Prediction completed in fold: 5


In [0]:
sub_file=sub.copy()
sub_file['target']=final_preds1[:,1]
sub_file.shape

(509, 2)

In [0]:
#Write to file
sub_file.to_csv('./submission_file_tta1.csv',index=False)